# Lab 10 - Module 3: Analysis and Visualization

**Time:** ~15-20 minutes

In this module, you'll visualize the patterns in your data to answer the core question:

> **Does AI confidence match actual accuracy?**

You'll create three visualizations:
1. **Confusion Matrix Heatmap** - How often confidence matched performance
2. **Error Rate Bar Chart** - Which prompt categories had the most errors
3. **Overconfidence Examples** - Specific cases where AI was confident but wrong

These visualizations will reveal patterns that might be invisible when looking at individual prompts.

## Setup: Import Libraries and Load Data

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, Markdown

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries loaded successfully!")

# Google Drive Setup

**IMPORTANT:** This module loads data from your Google Drive (saved in previous modules).

Run the cell below to mount Google Drive. You may need to authorize access again.

In [ ]:
from google.colab import drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
LAB_DIR = '/content/drive/MyDrive/DATA1010/Lab10'
print("Google Drive mounted successfully!")
print(f"Lab directory: {LAB_DIR}")

## Enter Your Group Code

Use the **same group code** from Modules 0, 1, and 2.

In [ ]:
group_code = int(input("Enter your group code: "))

# Load all data files
prompts_filename = f"{LAB_DIR}/lab10_group_{group_code}_prompts.csv"
predictions_filename = f"{LAB_DIR}/lab10_group_{group_code}_predictions.csv"
evaluations_filename = f"{LAB_DIR}/lab10_group_{group_code}_evaluations.csv"

try:
    prompts_df = pd.read_csv(prompts_filename)
    predictions_df = pd.read_csv(predictions_filename)
    evaluations_df = pd.read_csv(evaluations_filename)
    
    print(f"✓ Loaded prompts from {prompts_filename}")
    print(f"✓ Loaded predictions from {predictions_filename}")
    print(f"✓ Loaded evaluations from {evaluations_filename}")
    print(f"✓ Group Code: {group_code}")
    
except FileNotFoundError as e:
    print(f"❌ ERROR: Could not find required files")
    print(f"Missing: {e.filename}")
    print("\nMake sure you:")
    print("1. Ran Module 0 (to generate prompts)")
    print("2. Ran Module 1 (to collect predictions)")
    print("3. Ran Module 2 (to evaluate responses)")
    print("4. Are using the same group code!")

## Merge All Data

Combine prompts, predictions, and evaluations into one complete dataset.

In [ ]:
# Merge all dataframes
complete_df = prompts_df.merge(predictions_df, on=['prompt_id', 'category'])
complete_df = complete_df.merge(evaluations_df, on=['prompt_id', 'category'])

# Create simplified categories for visualization
def simplify_confidence(conf):
    if 'No caveats' in conf:
        return 'Confident'
    elif 'Refused' in conf:
        return 'Refused'
    else:
        return 'Cautious'

def simplify_accuracy(acc):
    if 'Fully accurate' in acc or 'Mostly accurate' in acc:
        return 'Accurate'
    elif 'Refused' in acc:
        return 'Refused'
    else:
        return 'Inaccurate'

complete_df['confidence_simple'] = complete_df['ai_confidence'].apply(simplify_confidence)
complete_df['accuracy_simple'] = complete_df['actual_accuracy'].apply(simplify_accuracy)

# Save complete dataset
complete_filename = f"{LAB_DIR}/lab10_group_{group_code}_complete.csv"
complete_df.to_csv(complete_filename, index=False)

print(f"✓ Merged {len(complete_df)} prompts")
print(f"✓ Saved complete dataset to {complete_filename}")
print("\nFirst few rows of your complete dataset:")
display(complete_df[['prompt_id', 'category_display', 'confidence_simple', 'accuracy_simple']].head())

## Visualization 1: Confusion Matrix Heatmap

This heatmap shows how AI's **expressed confidence** compares to **actual performance**.

**What to look for:**
- **Diagonal cells** (Confident+Accurate, Cautious+Inaccurate): Confidence matched reality
- **Off-diagonal cells** (Confident+Inaccurate): Overconfidence
- **Cell counts**: How many prompts fall into each combination

In [ ]:
# Create confusion matrix
ct = pd.crosstab(complete_df['confidence_simple'], 
                 complete_df['accuracy_simple'],
                 margins=False)

# Create heatmap
fig, ax = plt.subplots(figsize=(10, 7))
sns.heatmap(ct, annot=True, fmt='d', cmap='YlOrRd', 
            cbar_kws={'label': 'Count'}, 
            linewidths=2, linecolor='black',
            ax=ax)

plt.title('AI Confidence vs Actual Accuracy\n(Confusion Matrix)', 
          fontsize=16, fontweight='bold', pad=20)
plt.ylabel('AI Confidence Level', fontsize=14, fontweight='bold')
plt.xlabel('Actual Accuracy', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Calculate and display overconfidence rate
try:
    confident_total = complete_df[complete_df['confidence_simple'] == 'Confident'].shape[0]
    overconfident = complete_df[
        (complete_df['confidence_simple'] == 'Confident') & 
        (complete_df['accuracy_simple'] == 'Inaccurate')
    ].shape[0]
    
    if confident_total > 0:
        overconfidence_rate = (overconfident / confident_total) * 100
        print("="*60)
        print("KEY METRIC: OVERCONFIDENCE RATE")
        print("="*60)
        print(f"Confident responses: {confident_total}")
        print(f"Overconfident (Confident but Inaccurate): {overconfident}")
        print(f"\n🚨 OVERCONFIDENCE RATE: {overconfidence_rate:.1f}%")
        print("="*60)
        print()
        
        if overconfidence_rate > 50:
            print("⚠️ HIGH OVERCONFIDENCE: More than half of confident responses were inaccurate!")
        elif overconfidence_rate > 25:
            print("ℹ️ MODERATE OVERCONFIDENCE: About 1 in 4 confident responses were inaccurate.")
        elif overconfidence_rate > 0:
            print("✓ LOW OVERCONFIDENCE: Most confident responses were accurate.")
        else:
            print("✓ NO OVERCONFIDENCE: All confident responses were accurate!")
    else:
        print("Note: AI expressed no confident responses in your data.")
except:
    print("Could not calculate overconfidence rate (possibly no confident responses).")

## Visualization 2: Error Rates by Category

This bar chart shows which **prompt categories** had the highest error rates.

**What to look for:**
- **Tallest bars**: Categories where AI struggles most
- **Shortest bars**: Categories where AI performs well
- **Patterns**: Do citation requests have more errors than commonsense questions?

In [ ]:
# Calculate error rate by category
error_rates = complete_df.groupby('category_display').apply(
    lambda x: (x['accuracy_simple'] == 'Inaccurate').sum() / len(x) * 100
).sort_values(ascending=False)

# Create bar chart
fig, ax = plt.subplots(figsize=(12, 7))
bars = ax.bar(range(len(error_rates)), error_rates.values, 
              color='coral', edgecolor='black', linewidth=2)

# Add value labels on bars
for i, (bar, val) in enumerate(zip(bars, error_rates.values)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.0f}%',
            ha='center', va='bottom', fontweight='bold', fontsize=11)

ax.set_xticks(range(len(error_rates)))
ax.set_xticklabels(error_rates.index, rotation=45, ha='right')
ax.set_ylabel('Error Rate (%)', fontsize=14, fontweight='bold')
ax.set_xlabel('Prompt Category', fontsize=14, fontweight='bold')
ax.set_title('Error Rate by Prompt Category\n(Sorted from Highest to Lowest)', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_ylim(0, 110)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% threshold')

plt.tight_layout()
plt.show()

# Display summary
print("="*60)
print("ERROR RATES BY CATEGORY")
print("="*60)
for cat, rate in error_rates.items():
    print(f"{cat:25s}: {rate:5.1f}%")
print("="*60)
print(f"\nHardest category: {error_rates.index[0]} ({error_rates.values[0]:.1f}%)")
print(f"Easiest category: {error_rates.index[-1]} ({error_rates.values[-1]:.1f}%)")

## Visualization 3: Overconfidence Examples

This table shows **specific prompts** where AI was confident but inaccurate.

**What to look for:**
- What kinds of prompts trigger overconfident failures?
- What specific errors did the AI make?
- Were the errors subtle or obvious?

In [ ]:
# Filter for overconfident cases
overconfident_df = complete_df[
    (complete_df['confidence_simple'] == 'Confident') & 
    (complete_df['accuracy_simple'] == 'Inaccurate')
]

if len(overconfident_df) > 0:
    print("="*70)
    print(f"OVERCONFIDENCE EXAMPLES: {len(overconfident_df)} cases found")
    print("="*70)
    print()
    
    # Display table
    display_cols = ['prompt_id', 'category_display', 'prompt_text', 
                    'ai_confidence', 'actual_accuracy', 'error_types']
    
    display_df = overconfident_df[display_cols].copy()
    display_df.columns = ['ID', 'Category', 'Prompt', 'AI Confidence', 
                           'Actual Result', 'Error Types']
    
    # Style the table
    styled_html = display_df.to_html(index=False, escape=False, 
                                       classes='table table-striped table-bordered')
    display(HTML(styled_html))
    
    # Show detailed error information
    print("\nDETAILED ERROR DESCRIPTIONS:")
    print("="*70)
    for idx, row in overconfident_df.iterrows():
        print(f"\nPrompt #{row['prompt_id']}: {row['category_display']}")
        print(f"Prompt: {row['prompt_text']}")
        if pd.notna(row['error_details']) and row['error_details'].strip():
            print(f"Errors found: {row['error_details']}")
        else:
            print(f"Error types: {row['error_types']}")
        print("-" * 70)
else:
    print("="*70)
    print("NO OVERCONFIDENCE DETECTED")
    print("="*70)
    print("\nYour AI model did not show any cases where it was:")
    print("- Confident (no caveats) AND")
    print("- Inaccurate (major errors or hallucinations)")
    print()
    print("This could mean:")
    print("1. The AI was well-calibrated on your prompts")
    print("2. The AI expressed caution on difficult prompts")
    print("3. The AI was generally accurate")
    print()
    print("Check the confusion matrix above for the full picture!")

## Summary Statistics

Key metrics about your group's data.

In [ ]:
# Calculate comprehensive statistics
total_prompts = len(complete_df)
accurate_count = (complete_df['accuracy_simple'] == 'Accurate').sum()
inaccurate_count = (complete_df['accuracy_simple'] == 'Inaccurate').sum()
refused_count = (complete_df['accuracy_simple'] == 'Refused').sum()

confident_count = (complete_df['confidence_simple'] == 'Confident').sum()
cautious_count = (complete_df['confidence_simple'] == 'Cautious').sum()
refused_conf_count = (complete_df['confidence_simple'] == 'Refused').sum()

# Calibration metrics
well_calibrated = complete_df[
    ((complete_df['confidence_simple'] == 'Confident') & (complete_df['accuracy_simple'] == 'Accurate')) |
    ((complete_df['confidence_simple'] == 'Cautious') & (complete_df['accuracy_simple'] == 'Inaccurate'))
].shape[0]

overconfident_count = complete_df[
    (complete_df['confidence_simple'] == 'Confident') & 
    (complete_df['accuracy_simple'] == 'Inaccurate')
].shape[0]

underconfident_count = complete_df[
    (complete_df['confidence_simple'] == 'Cautious') & 
    (complete_df['accuracy_simple'] == 'Accurate')
].shape[0]

# Display summary
print("="*70)
print("COMPREHENSIVE SUMMARY STATISTICS")
print("="*70)
print(f"\nTotal prompts tested: {total_prompts}")
print()
print("ACCURACY BREAKDOWN:")
print(f"  Accurate responses:   {accurate_count:2d} ({accurate_count/total_prompts*100:5.1f}%)")
print(f"  Inaccurate responses: {inaccurate_count:2d} ({inaccurate_count/total_prompts*100:5.1f}%)")
print(f"  Refused responses:    {refused_count:2d} ({refused_count/total_prompts*100:5.1f}%)")
print()
print("CONFIDENCE BREAKDOWN:")
print(f"  Confident (no caveats): {confident_count:2d} ({confident_count/total_prompts*100:5.1f}%)")
print(f"  Cautious (with caveats): {cautious_count:2d} ({cautious_count/total_prompts*100:5.1f}%)")
print(f"  Refused:                {refused_conf_count:2d} ({refused_conf_count/total_prompts*100:5.1f}%)")
print()
print("CALIBRATION ANALYSIS:")
print(f"  Well-calibrated (confidence matched accuracy): {well_calibrated:2d} ({well_calibrated/total_prompts*100:5.1f}%)")
print(f"  Overconfident (confident but wrong):          {overconfident_count:2d} ({overconfident_count/total_prompts*100:5.1f}%)")
print(f"  Underconfident (cautious but right):          {underconfident_count:2d} ({underconfident_count/total_prompts*100:5.1f}%)")
print("="*70)

# Interpretation
print("\nINTERPRETATION:")
if overconfident_count > underconfident_count:
    print("→ Your AI showed MORE overconfidence than underconfidence.")
    print("  This means it was more likely to sound sure when wrong than")
    print("  to sound uncertain when right.")
elif underconfident_count > overconfident_count:
    print("→ Your AI showed MORE underconfidence than overconfidence.")
    print("  This means it was more likely to sound cautious when right than")
    print("  to sound sure when wrong.")
else:
    print("→ Your AI showed EQUAL amounts of overconfidence and underconfidence.")

if well_calibrated / total_prompts > 0.75:
    print("\n→ Overall, the AI was fairly WELL-CALIBRATED (>75% match).")
elif well_calibrated / total_prompts > 0.5:
    print("\n→ The AI showed MODERATE calibration (50-75% match).")
else:
    print("\n→ The AI showed POOR calibration (<50% match).")
    print("  Confidence did not reliably predict accuracy!")

## Export Summary for Cross-Group Comparison

Save your key metrics for instructor-led cross-group discussion.

In [ ]:
# Create summary dict
summary = {
    'group_code': group_code,
    'total_prompts': total_prompts,
    'accurate_count': accurate_count,
    'inaccurate_count': inaccurate_count,
    'confident_count': confident_count,
    'overconfident_count': overconfident_count,
    'underconfident_count': underconfident_count,
    'overconfidence_rate': (overconfident_count / confident_count * 100) if confident_count > 0 else 0,
    'calibration_rate': (well_calibrated / total_prompts * 100)
}

# Save summary
import json
summary_filename = f"{LAB_DIR}/lab10_group_{group_code}_summary.json"
with open(summary_filename, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Summary saved to {summary_filename}")
print("\nShare these metrics with other groups for comparison!")

## Questions for Module 3

Answer these questions on your lab handout using the visualizations and statistics above.

### Q13: Confusion Matrix Interpretation

Looking at the confusion matrix heatmap, how often did the AI's self-assessment (expressed confidence) match its actual performance?

*(Answer on your handout)*

### Q14: Overconfidence vs Underconfidence

Did the AI show **overconfidence** (confident tone but inaccurate) or **underconfidence** (cautious but accurate)? Which pattern was more common in your data?

*(Answer on your handout)*

### Q15: Category Analysis

Which prompt category had the highest error rate in the bar chart? Why do you think this category is difficult for AI models?

*(Answer on your handout)*

### Q16: Overconfidence Example Analysis

Looking at the overconfidence examples table, pick your group's most "overconfident" prompt (confident but very wrong). What made the AI fail despite sounding sure of itself?

*(Answer on your handout)*

### Q17: Uncertainty Correlation

Did expressing uncertainty (caveats like "I might be wrong") correlate with lower accuracy? Use your data to support your answer.

*(Answer on your handout)*

### Q18: Cross-Group Comparison

(Cross-group comparison) Compare your confusion matrix or overconfidence rate with another group's results. Did different groups find similar patterns of overconfidence?

*(Answer on your handout)*

## Next Steps

1. **Answer Q13-Q18** on your lab handout
2. **Discuss findings** with other groups (if time permits)
3. **Continue to Module 4** for synthesis and real-world implications
4. **Use the same group code** in Module 4

In Module 4, you'll connect these patterns to real-world AI failures and develop best practices for responsible AI use!